In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
PATH = '/home/disk/orca/adaley17/Research/Stress_Separation/'
with open(PATH + 'lsdiaga_1982_2022_sat_ts_5day.txt', 'r') as file:
    # content = file.read()
    # print(content)
    # print(file.readline())
    file.seek(0)
    for i, line in enumerate(file):
        if 'LAST' in line:
            print(f"Line {i}: {line}")
    # lines_with_AL = [line for line in file if 'LAST' in line]



## Get the Column Names

In [55]:
num_entries = 139
column_names = [np.nan] * num_entries

with open(PATH + 'lsdiaga_1982_2022_sat_ts_5day.txt', 'r') as file:
    # Move the file pointer to the beginning of the file
    file.seek(0)

    # Read lines 0 to 138
    lines = [file.readline().split() for _ in range(139)]

    for i in range(len(lines)):
        # print(lines[i][-1])
        column_names[i] = lines[i][-1]

In [ ]:
lines_with_LAT = [line for line in lines if line[-1] == 'LAT']
for i, line in enumerate(lines_with_LAT):
    print(f"Line {i}: {line}")
    # lat_data = np.array([float(item) if item != '9999' else np.nan for sublist in lines_with_LAT for item in sublist if item != 'LAT'])
    # lat_data = np.array([float(line[0]) for line in lines_with_LAT])

In [4]:
PATH = '/home/disk/orca/adaley17/Research/Stress_Separation/'
with open(PATH + 'lsdiaga_1982_2022_sat_ts_5day.txt', 'r') as file:
    # Move the file pointer to the beginning of the file
    file.seek(0)
    # file.readline().split()
    # Read lines 0 to 138
    lines = [file.readline().split() for _ in range(1925149)]
    # lines.split()

    # # Combine the lines into one line
    # combined_line = ''.join(lines)

    # # Print the combined line
    # print(combined_line)

In [10]:
def extract_data_of_interest(start, lines, data_of_interest):
    count=0
    for i in range(start, len(lines), 139):
        # print(i, lines[i])
        for j in range(len(lines[i])-1): #Here we are skipping the last element of the line which is the variable name
            data_of_interest[count:,] = lines[i][j]
            count+=1

        return data_of_interest;

def extract_names_and_codes(start, lines, data, choice):

    #Choice is the index of the variable name we want to extract, 0 for names and -2 for codes
    count=0 #Counter variable
    for i in range(start, len(lines), 139):

        names_and_codes = [lines[i][choice]] * 22 #every time period has 22 predictions but the storm name is the same for all of them
        data[count*22:(count+1)*22, :] = np.array(names_and_codes).reshape(-1, 1) #Reshape the array to be a column vector and assign it to the name array

        count+=1

    return data

In [38]:
size = 13850 * 22 #Number of time periods * number of predictions per time period

times = np.full((size, 1), np.nan)
lat = np.full((size, 1), np.nan)
lon = np.full((size, 1), np.nan)
vmax = np.full((size, 1), np.nan)
mslp = np.full((size, 1), np.nan)
nature = np.full((size, 1), np.nan)
delta_intensity = np.full((size, 1), np.nan)
name = np.full((size, 1), 'np.nan')
code = np.full((size, 1), 'np.nan')
daily_sst_avg = np.full((size, 1), np.nan)
mid_level_rh = np.full((size, 1), np.nan)
vshear = np.full((size, 1), np.nan)
vert_vel = np.full((size, 1), np.nan)

# Extracting Data of interest
name = extract_names_and_codes(0, lines,name, 0).astype(object)
code = extract_names_and_codes(0, lines, code, -2).astype(object)
times = extract_data_of_interest(1, lines, times).astype(object)
vmax = extract_data_of_interest(2, lines, vmax).astype(object)
mslp = extract_data_of_interest(3, lines, mslp).astype(object)
nature = extract_data_of_interest(4, lines, nature).astype(object)
delta_intensity = extract_data_of_interest(5, lines, delta_intensity).astype(object)
lat = extract_data_of_interest(8, lines, lat).astype(object)
lon = extract_data_of_interest(9, lines, lon).astype(object)

#Fix here
# daily_sst_avg = extract_data_of_interest(19, lines, daily_sst_avg)
# mid_level_rh = extract_data_of_interest(29, lines, mid_level_rh)
vshear = extract_data_of_interest(64, lines, vshear).astype(object)
vert_vel = extract_data_of_interest(67, lines, vert_vel).astype(object)

#Changing 9999 to nans and didvide by 10 to get the correct values
vmax[vmax==9999] = np.nan
vmax = vmax / 10

mslp[mslp == 9999] = np.nan

nature[nature == 9999] = np.nan
nature[nature == 0] = 'Wave'
nature[nature == 1] = 'Tropical'
nature[nature == 2] = 'Subtropical'
nature[nature == 3] = 'Extratropical'

vshear[vshear == 9999] = np.nan
vshear = vshear / 10

vert_vel[vert_vel == 9999] = np.nan
vert_vel = vert_vel / 100



# Create a DataFrame with the extracted data
data = {
    'Name': name.flatten(),
    'Code': code.flatten(),
    'Times': times.flatten(),
    'Latitude': lat.flatten(),
    'Longitude': lon.flatten(),
    'Vmax': vmax.flatten(),
    'MSLP': mslp.flatten(),
    'Nature': nature.flatten(),
    'Delta_Intensity': delta_intensity.flatten(),
    # 'Daily_SST_Avg': daily_sst_avg.flatten(),
    # 'Mid_Level_RH': mid_level_rh.flatten(),
    'Vshear': vshear.flatten(),
    'Vert_Vel': vert_vel.flatten()
}

df = pd.DataFrame(data)
print(df.head())




   Name    Code Times Latitude Longitude Vmax    MSLP    Nature  \
0  ALBE  AL0119 -12.0   9999.0    9999.0  NaN     NaN       NaN   
1  ALBE  AL0119  -6.0   9999.0    9999.0  NaN     NaN       NaN   
2  ALBE  AL0119   0.0    217.0     871.0  2.0  1005.0  Tropical   
3  ALBE  AL0119   6.0    222.0     865.0  2.5  1004.0  Tropical   
4  ALBE  AL0119  12.0    226.0     858.0  3.0  1003.0  Tropical   

  Delta_Intensity Vshear Vert_Vel  
0             1.0   29.6    12.89  
1             0.0   30.2     7.71  
2             0.0   29.8     8.56  
3             0.0   36.2    10.45  
4             0.0   34.0     8.08  


In [40]:
df.tail()

,Name,Code,Times,Latitude,Longitude,Vmax,MSLP,Nature,Delta_Intensity,Vshear,Vert_Vel
304695,NICO,AL1720,120.0,9999.0,9999.0,NaN,NaN,NaN,0.0,NaN,NaN
304696,NICO,AL1720,120.0,9999.0,9999.0,NaN,NaN,NaN,0.0,NaN,NaN
304697,NICO,AL1720,120.0,9999.0,9999.0,NaN,NaN,NaN,0.0,NaN,NaN
304698,NICO,AL1720,120.0,9999.0,9999.0,NaN,NaN,NaN,0.0,NaN,NaN
304699,NICO,AL1720,120.0,9999.0,9999.0,NaN,NaN,NaN,0.0,NaN,NaN


In [7]:
lat.shape

(304700, 1)

In [6]:
times = np.full((len(lines[1])-1, 1), 'np.nan')
vmax = np.full((len(lines[2])-1, 1), 'np.nan')
mslp = np.full((len(lines[3])-1, 1), 'np.nan')
nature = np.full((len(lines[4])-1, 1), 'np.nan')
delta_intensity = np.full((len(lines[5])-1, 1), 'np.nan')
lat = np.full((len(lines[8])-1, 1), 'np.nan')
lon = np.full((len(lines[9])-1, 1), 'np.nan')
daily_sst_avg = np.full((len(lines[19])-1, 1), 'np.nan')
mid_level_rh = np.full((len(lines[29])-1, 1), np.nan)

shear = np.full((len(lines[64])-1, 1), np.nan)
""" Generalized 850-200 hPa shear magnitude (kt *10) vs time (takes into account all levels from 1000 to 100 hPa) """ 
vert_vel = np.full((len(lines[67])-1, 1), np.nan)


# Times
for i in range(len(lines[1])-1):
    times[i,:] = lines[1][i]

# Vmax
for i in range(len(lines[2])-1):
    vmax[i,:] = lines[2][i]

# MSLP
for i in range(len(lines[3])-1):
    mslp[i,:] = lines[3][i]

# Nature
for i in range(len(lines[4])-1):
    nature[i,:] = lines[4][i]

# Delta Intensity
for i in range(len(lines[5])-1):
    delta_intensity[i,:] = lines[5][i]

# Latitude
for i in range(len(lines[8])-1):
    lat[i,:] = lines[8][i]

# Longitude
for i in range(len(lines[9])-1):
    lon[i,:] = lines[9][i]

# Daily SST Avg
for i in range(len(lines[19])-1):
    # print(i)
    daily_sst_avg[i,:] = lines[19][i]

# Mid Level RH
for i in range(len(lines[29])-1):
    mid_level_rh[i,:] = float(lines[29][i])

# Shear
for i in range(len(lines[64])-1):
    shear[i,:] = float(lines[64][i])/10

# Vert Vel
for i in range(len(lines[67])-1):
    vert_vel[i,:] = float(lines[67][i])/100


    

In [46]:
### Mu-Chieh Ko,Xiaomin Chen, Miroslav Kubat and Sundararaman Gopalakrishnan
delv = np.full((len(lines[6])-1, 1), np.nan)
incv = np.full((len(lines[7])-1, 1), np.nan)
lat = np.full((len(lines[8])-1, 1), np.nan)
lon = np.full((len(lines[9])-1, 1), np.nan)
dist_to_land = np.full((len(lines[14])-1, 1), np.nan)
daily_sst_avg = np.full((len(lines[19])-1, 1), np.nan)
mid_level_rh = np.full((len(lines[29])-1, 1), np.nan)
div_200 = np.full((len(lines[33])-1, 1), np.nan)
rad_max_wind = np.full((len(lines[-7])-1, 1), np.nan) #Needs further investifgation since the brightness temp is used
shear = np.full((len(lines[64])-1, 1), np.nan) #Generalized 850-200 hPa shear magnitude (kt *10) vs time (takes into account all levels from 1000 to 100 hPa) """ 

#ShearS
#ShdirS
#ShearM
#ShdirM
#ShearD
#ShdirD
#CAPE_US
#CAPE_DS
# HLCY_US
# HLCY_DS
#Convergence
#Temperature Difference between core and environment
# Wavenumber1




# Delta V
for i in range(len(lines[6])-1):
    delv[i,:] = float(lines[6][i])

# Inc V
for i in range(len(lines[7])-1):
    incv[i,:] = float(lines[7][i])

# Latitude
for i in range(len(lines[8])-1):
    lat[i,:] = float(lines[8][i])

# Longitude
for i in range(len(lines[9])-1):
    lon[i,:] = float(lines[9][i])

# Distance to Land
for i in range(len(lines[14])-1):
    dist_to_land[i,:] = float(lines[14][i])
    dist_to_land[dist_to_land == 9999.] = np.nan
    
# Daily SST Avg
for i in range(len(lines[19])-2):
    daily_sst_avg[i,:] = float(lines[19][i])
    daily_sst_avg[daily_sst_avg == 9999.] = np.nan

# Mid Level RH
for i in range(len(lines[29])-1):
    mid_level_rh[i,:] = float(lines[29][i])
    mid_level_rh[mid_level_rh == 9999.] = np.nan

# Div 200
for i in range(len(lines[33])-1):
    div_200[i,:] = float(lines[33][i])
    div_200[div_200 == 9999.] = np.nan

# Shear
for i in range(len(lines[64])-1):
    shear[i,:] = float(lines[64][i])/10 #Convert to m/s

# Rad Max Wind
for i in range(len(lines[-7])-1):
    rad_max_wind[i,:] = float(lines[-7][i])
    rad_max_wind[rad_max_wind == 9999.] = np.nan



In [47]:
div_200

array([[ 64.],
       [ 65.],
       [ 84.],
       [ 70.],
       [ 51.],
       [114.],
       [ 42.],
       [ 27.],
       [ -7.],
       [  8.],
       [-17.],
       [-37.],
       [-48.],
       [-52.],
       [-25.],
       [-32.],
       [-41.],
       [ nan],
       [ nan],
       [ nan],
       [ nan]])

In [32]:
# Extract the substring
substring = combined_line[0:46]

# Split the combined_line into a list of strings
combined_line_list = combined_line.split()

# Find the index of the substring in the list
start_index = combined_line_list.index(substring)

# Print every 4th string after the substring
for i in range(start_index, len(combined_line_list), 4):
    print(combined_line_list[i])

ValueError: ' ALBE 820602 12   20   21.7   87.1 1005 AL0119' is not in list

In [34]:
combined_line_list[-1]

'LAST'

In [ ]:
import xarray as xr

# Assuming you have an xarray DataArray named 'data_array'
nan_indices = np.argwhere(data_array.isnull().values)
print(nan_indices)